In [1]:
import sys
import numpy as np
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from transformers import EarlyStoppingCallback
import json
import pandas as pd
import os
import re
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = Config(executors=[ThreadPoolExecutor()])
parsl.load(config)

In [3]:
def extractName(text):
  return re.findall(r'"name"\s*:\s*"([^"]+)"',text)

def extractTag(text):
  matches = re.findall(r'\"tags\": \[(.*?)\]', text, re.DOTALL)
  matches = re.findall(r'\"(.*?)\"', matches[0].strip(','))
  return matches

def extractDescription(text):
  return re.findall(r'"description"\s*:\s*"([^"]+)"',text)

def extractLabel(text):
    return re.findall(r'"label"\s*:\s*"([^"]+)"', text)

def removeWords(text, words):
  for word in words:
    text = text.replace(word,"")
  return text

In [4]:
os.chdir(os.path.join(os.pardir,os.pardir, 'workflows_galaxy'))
nomes_arquivos = []
diretorio = os.getcwd()
print(diretorio)
for item in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, item)
    if os.path.isfile(caminho_completo):
        nomes_arquivos.append(item)

print(len(nomes_arquivos))


/home/lyncoln/Git/similaridade_workflow/workflows_galaxy
1014


In [5]:
dic_workflows = {}
@python_app
def processar_arquivo(arquivo):
    # Carregar o arquivo JSON
    with open(arquivo, 'r') as file:
        fileName = file.name
        dados = file.read()
        # print(dados)
        matches_name = extractName(dados)
        matches_tag = extractTag(dados)
        matches_description = extractDescription(dados)
        matches_label = extractLabel(dados)
        combined_results = ' '.join(matches_name + matches_tag + matches_description + matches_label)
        # combined_results = removeWords(combined_results,['input dataset(s)'])
        dic_workflows[fileName] = combined_results

        save_path = 'crawler_results'
        os.makedirs(save_path, exist_ok=True)  # Cria a pasta se não existir
        output_file_path = os.path.join(save_path, f"{fileName}.txt")

        # Salvando os resultados em um arquivo
        with open(output_file_path, 'w') as output_file:
            output_file.write(combined_results)

        return combined_results


# Inicia o processamento dos arquivos em paralelo
futures = [processar_arquivo(arquivo) for arquivo in nomes_arquivos]

resultados = [future.result() for future in futures]

In [6]:
# texto_completo = " ".join(dic_workflows.keys()) + " " + " ".join(dic_workflows.values())
# palavras = texto_completo.split()
# frequencias = Counter(palavras)
# df = pd.DataFrame(list(frequencias.items()), columns=['Palavra', 'Frequência'])
# df = df.sort_values(by='Frequência', ascending=False)
# print(df[0:50])

In [7]:
df = pd.DataFrame(list(dic_workflows.items()), columns=['name', 'text'])
print(df)

                       name                                               text
0     0c86c39dcd9e08c6.json                                         Project_CP
1     692a2b0bb818336d.json  Workflow constructed from history 'K22063917 A...
2     3680984663c813e1.json  dhfr Bowtie2 output bowtie2 MPileup output_mpi...
3     49f8b32c3206f76c.json  Workflow for Genomic Data Science with Galaxy ...
4     25adc55d2a26e34b.json  GigaScience Example 1B -- aye-aye FST (importe...
...                     ...                                                ...
1009  d5cba8a5ba6880fd.json  BRACA2 - primer design exon Input dataset snp ...
1010  e0da87cadb1e6d5f.json  2 peaks-result data file 2 and more peaks 16 c...
1011  fbf75fbb72b488bd.json  'BBL735_Lab2(Olympic)_AT' olympics.tsv Input d...
1012  0d8eb75d28fa7df2.json  Desanka Lazic_Project Coriell-NA12880_R2.fastq...
1013  f8238234db6f04c3.json  handson Input dataset Input dataset Join outpu...

[1014 rows x 2 columns]


In [8]:
# Carregar o modelo e o tokenizer
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
model = BertModel.from_pretrained('google-bert/bert-large-uncased')

texts = df['text'].tolist()

# Função para converter texto em embedding
def text_to_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().numpy()

# Converter todos os textos para embeddings
embeddings = [text_to_embedding(text) for text in texts]

# Converter a lista de embeddings em um array 2D
embeddings_array = np.vstack(embeddings)

# Calcular a matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Identificar os índices dos três textos mais similares para cada texto
similar_indices = similarity_matrix.argsort(axis=1)[:, :-5:-1]  # -4 porque os três mais similares excluem o próprio texto

# Remover o índice do próprio texto
corrected_similar_indices = []
for idx, indices in enumerate(similar_indices):
    filtered_indices = [index for index in indices if index != idx][:3]  # Exclui o próprio e pega os três mais similares
    corrected_similar_indices.append(filtered_indices)

# Criar coluna no DataFrame para os índices dos textos mais similares
df['top3_description'] = corrected_similar_indices

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
# Carregar o modelo e o tokenizer
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

texts = df['text'].tolist()

# Função para converter texto em embedding
def text_to_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().numpy()

# Converter todos os textos para embeddings
embeddings = [text_to_embedding(text) for text in texts]

# Converter a lista de embeddings em um array 2D
embeddings_array = np.vstack(embeddings)

# Calcular a matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Identificar os índices dos três textos mais similares para cada texto
similar_indices = similarity_matrix.argsort(axis=1)[:, :-5:-1]  # -4 porque os três mais similares excluem o próprio texto

# Remover o índice do próprio texto
corrected_similar_indices = []
for idx, indices in enumerate(similar_indices):
    filtered_indices = [index for index in indices if index != idx][:3]  # Exclui o próprio e pega os três mais similares
    corrected_similar_indices.append(filtered_indices)

# Criar coluna no DataFrame para os índices dos textos mais similares
df['top3_description_scibert'] = corrected_similar_indices

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
os.chdir(os.pardir)
df.to_csv("top3_descricao.csv")